In [ ]:
import time 
import pandas as pd 
import numpy as np 

from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

def sele_daily_craw():
    url = 'https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_COVID19_005_D'

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    browser = webdriver.Chrome('chromedriver', chrome_options=options) # 같은 디렉토리 내에 chtromdriver.exe
    browser.implicitly_wait(1)

    browser.get(f'{url}')
    browser.implicitly_wait(1)

    res = browser.find_element_by_css_selector('#mainTable > tbody')

    text_list = [elem.strip() for elem in res.text.strip().split('\n')] 

    n = len(text_list)
    result = list() 
    result.append(text_list[:32][1:12] )

    idx = 32
    for i in range(1, 19): 
        if i == 1: 
            result.append( text_list[idx:idx+32][1:12] )
            idx += 32
        else: 
            result.append( text_list[idx:idx+31][:11]  )
            idx += 31

    print("\n\n")
    for elem in result:
        elem
        
    col_list= ['area','overseas','domestic','confirmed','confirmed_c','iso','iso_c','iso_rel','iso_rel_c','death','death_c']
    daily_area_df = pd.DataFrame(result, columns=col_list)
    
    n = len(text_list)
    result = list() 
    result.append(text_list[591:][1:12] )

    idx = 591
    for i in range(1, 3):
        if i == 1:
            idx += 32
        else:
            result.append(text_list[idx:idx+31][:11])
            idx += 31

    for elem in result:
        elem
            
    col_list= ['gender','overseas','domestic','confirmed','confirmed_c','iso','iso_c','iso_rel','iso_rel_c','death','death_c']
    daily_gender_df = pd.DataFrame(result, columns=col_list)
    
    n = len(text_list)
    result = list() 
    result.append(text_list[654:][1:12] )

    idx = 654
    for i in range(1, 10): 
        if i == 1: 
            idx += 32
        else: 
            result.append( text_list[idx:idx+31][:11]  )
            idx += 31

    for elem in result:
        elem
        
    col_list= ['age','overseas','domestic','confirmed','confirmed_c','iso','iso_c','iso_rel','iso_rel_c','death','death_c']
    daily_age_df = pd.DataFrame(result, columns=col_list)
    
    
    daily_area_new = daily_area_df[['area', 'overseas', 'domestic','confirmed', 'iso','iso_rel','death']]

    
    daily_area_cum = daily_area_df[['area', 'confirmed_c', 'iso_c','iso_rel_c','death_c']]
    daily_area_cum = daily_area_cum.rename({'area':'attr'}, axis='columns')
    daily_area_cum['type'] = 'area'

    daily_gender_new = daily_gender_df[['gender', 'overseas', 'domestic','confirmed', 'iso','iso_rel','death']]
    

    daily_gender_cum = daily_gender_df[['gender', 'confirmed_c', 'iso_c','iso_rel_c','death_c']]
    daily_gender_cum = daily_gender_cum.rename({'gender':'attr'}, axis='columns')
    daily_gender_cum['type'] = 'gender'
    

    daily_age_new = daily_age_df[['age', 'overseas', 'domestic','confirmed', 'iso','iso_rel','death']]
    

    daily_age_cum = daily_age_df[['age', 'confirmed_c', 'iso_c','iso_rel_c','death_c']]
    daily_age_cum = daily_age_cum.rename({'age':'attr'}, axis='columns')
    daily_age_cum['type'] = 'age'
    
    
    df_cum = pd.concat([daily_area_cum, daily_gender_cum])
    df_cum = pd.concat([df_cum, daily_age_cum])
    df_cum = df_cum[['type', 'attr', 'confirmed_c', 'iso_c','iso_rel_c','death_c']]
    df_cum = df_cum.rename({'confirmed_c':'confirmed','iso_c':'iso','iso_rel_c':'iso_rel','death_c':'death'}, axis='columns')
    
    ##mongodb 연결
    def conn_db():
        
        user = os.getenv("DBUSER", 'signature')
        pwd = os.getenv("DBPWD", 'shanekang')
        ip_addr = os.getenv("DBADDR", '54.180.213.105')
        conn = MongoClient(f'mongodb://{user}:{pwd}@{ip_addr}:27017') 
        return conn
    
    
    ##mongodb에 데이터 insert
    conn = conn_db() 

    area_daily_collection = conn.KORDAILY.area
    gender_daily_collection = conn.KORDAILY.gender
    age_daily_collection = conn.KOR_DAILY.age
    cum_daily_collection = conn.KOR_DAILY.cum

    print(datetime.now())
    print("crawling_area_daily Updated: ", insert_data(daily_area_new, area_daily_collection, 'date', 'area'))
    print("crawling_gender_daily Updated: ", insert_data(daily_gender_new, gender_daily_collection, 'date', 'gender'))
    print("crawling_age_daily Updated: ", insert_data(daily_age_new, age_daily_collection, 'date', 'age'))
    print("crawling_cum_daily Updated: ", insert_data(df_cum, cum_daily_collection, 'date', 'attr'))
    conn.close()
    
    

# sele_daily_craw()